In [2]:

# Import SparkSession
import os
from pyspark.sql import SparkSession

path_dir = os.path.join(os.getcwd(), "Documents", "Github", "streaming_dwh", "data", "Product")

# Create SparkSession 
spark = SparkSession \
      .builder \
      .master("local[1]") \
      .appName("Streaming DWH") \
      .getOrCreate()


In [3]:
test_df = spark.read.json("data/Product/1707844682350206.json").show()

+-----------------+----+-----------------+--------------+-------------+----------------+--------------------+--------------+-----+----------+------+
|         category|cogs|contains_caffeine|contains_fruit|contains_nuts|contains_veggies|          event_time|          item|price|product_id|  size|
+-----------------+----+-----------------+--------------+-------------+----------------+--------------------+--------------+-----+----------+------+
|Classic Smoothies| 1.5|            false|          true|        false|           false|2024-02-13 17:18:...|Sunrise Sunset| 4.99|      CS01|24 oz.|
+-----------------+----+-----------------+--------------+-------------+----------------+--------------------+--------------+-----+----------+------+



In [ ]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
streaming_df = spark.readStream\
    .format("json") \
    .option("maxFilesPerTrigger", 1) \
    .load("data/Product/")

streaming_df.printSchema()

In [30]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

streaming_df = spark.readStream\
    .format("json") \
    .option("maxFilesPerTrigger", 1) \
    .load("data/Product/")

# query = streaming_df \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()

bronzePath = "data/bronze/Product"

query = streaming_df \
    .writeStream \
    .outputMode("append") \
    .format("json") \
    .option("path", bronzePath) \
    .option("checkpointLocation", bronzePath + "/_checkpoint") \
    .queryName("bronzeIngestion") \
    .start()

query.awaitTermination()

# query.awaitTermination()

# query = (streaming_df
#   .writeStream.format("delta")
#   .option("checkpointLocation", bronzePath + "/_checkpoint")
#   .queryName("bronzeIngestion")
#   .outputMode("append") 
#   .start(bronzePath)
#   .awaitTermination())


In [8]:
# print("Input path:", path_dir)
# print(os.listdir(path=path_dir))

# Define schema
schema = """
    event_time TIMESTAMP, 
    product_id STRING, 
    category STRING, 
    item STRING, 
    size STRING, 
    cogs FLOAT, 
    price FLOAT, 
    contains_fruit BOOLEAN, 
    contains_veggies BOOLEAN, 
    contains_nuts BOOLEAN, 
    contains_caffeine BOOLEAN
"""

# Create the streaming_df to read from input directory
query = (
    spark.readStream
    .format("json")
    .schema(schema)
    .option("maxFilesPerTrigger", 1)
    .load("data/Product/")
    .writeStream
    .format("delta")
    .option("path", "bronze/Product")
    .outputMode("append")
    .start()
)

query.awaitTermination()

# streaming_df.display()

Py4JJavaError: An error occurred while calling o90.start.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.streaming.DataStreamWriter.startInternal(DataStreamWriter.scala:370)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:251)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.

In [5]:
 # Start running the query that prints the running counts to the console

# wordCounts = streaming_df.groupBy("category").count()

# query = wordCounts \
#     .writeStream \
#     .outputMode("complete") \
#     .format("console") \
#     .start()

# output_path = os.path.join(os.getcwd(), "data", "bronze", "Product")
# print("Output path:", output_path)

query = streaming_df  \
        .writeStream \
        .format("delta") \
        .option("path", "bronze/Product") \
        .queryName("bronze_ingestion") \
        .outputMode("append") \
        .option("checkpointLocation", "bronze/chkPoint") \
        .start()

query.awaitTermination()

Py4JJavaError: An error occurred while calling o45.start.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.streaming.DataStreamWriter.startInternal(DataStreamWriter.scala:370)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:251)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.